In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 6, 11)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,20,S,TOP,3000,60.5,0,5pct,SET50,2022-03-03


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,TOP,2.0000,0.0000,0.6000,0.0000,2.6000,"3,000",2022-03-03,2022-04-25,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'TOP'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,9555,TOP,61.50,42.50,I,0.00,60.50,817.89,1.22,57.50,"3,000",-11,11,"3,000",0,0,5pct,SET50


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

20

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,ASK,3000,37.75,0,6L,SET
1,O,BBL,2400,125.00,0,1L,SET50
2,O,BCPG,12000,11.10,0,MinP,SET100
3,O,GLOBAL,9000,20.60,0,1L,SET50
4,O,GULF,6000,46.00,0,1L,SET50
5,O,IVL,3000,48.00,0,P50%,SET50
6,O,KBANK,1500,142.00,0,5pct,SET50
7,O,KTC,2100,59.00,0,P25%,SET50
8,O,SINGER,3000,49.00,0,P25%,SET100
9,O,SSP,10000,9.35,0,6L,SET


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,NER,9000,7.10,0,P50%,SET
1,I,KCE,1000,68.00,0,C62.75,SET50
2,I,RCL,4000,47.50,0,CP1S,SET100
3,I,MAKRO,1500,39.00,0,6M,SET
4,I,BDMS,1500,27.50,0,RD05pct,SET50
5,I,ORI,15000,11.60,0,5pct,SET100
6,I,SIS,3600,30.00,0,5pct,SET
7,I,TTB,100000,1.33,0,5pct,SET50
8,I,TOP,3000,60.50,0,5pct,SET50


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(20, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
I,9,9,9,9,9,9
O,11,11,11,11,11,11


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
O,11,11,11,11,11,11
S,9,9,9,9,9,9


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(87, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(91, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
2,O,BCPG,12000.0,11.10,0.0,MinP,SET100,NaT,left_only
8,O,SINGER,3000.0,49.00,0.0,P25%,SET100,NaT,left_only
9,O,SSP,10000.0,9.35,0.0,6L,SET,NaT,left_only
10,O,STANLY,600.0,172.00,0.0,P25%,SET,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
O,11,11,11,11,11,11,11
S,9,9,9,9,9,9,9


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,ASK,3000,37.75,0,6L,SET,2022-03-03
1,O,BBL,2400,125.00,0,1L,SET50,2022-04-22
2,O,BCPG,12000,11.10,0,MinP,SET100,NaT
3,O,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
4,O,GULF,6000,46.00,0,1L,SET50,2022-03-04
5,O,IVL,3000,48.00,0,P50%,SET50,2022-05-31
6,O,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
7,O,KTC,2100,59.00,0,P25%,SET50,2022-04-18
8,O,SINGER,3000,49.00,0,P25%,SET100,NaT
9,O,SSP,10000,9.35,0,6L,SET,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
15,S,BDMS,1500,27.50,0,RD05pct,SET50,2022-03-09
12,S,KCE,1000,68.00,0,C62.75,SET50,2022-03-21
14,S,MAKRO,1500,39.00,0,6M,SET,2022-03-03
11,S,NER,9000,7.10,0,P50%,SET,2022-04-19
16,S,ORI,15000,11.60,0,5pct,SET100,2022-05-09
13,S,RCL,4000,47.50,0,CP1S,SET100,2022-03-23
17,S,SIS,3600,30.00,0,5pct,SET,2022-03-10
19,S,TOP,3000,60.50,0,5pct,SET50,2022-03-03
18,S,TTB,100000,1.33,0,5pct,SET50,2022-04-21


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
0,O,ASK,3000,37.75,0,6L,SET,2022-03-03
1,O,BBL,2400,125.00,0,1L,SET50,2022-04-22
2,O,BCPG,12000,11.10,0,MinP,SET100,NaT
3,O,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
4,O,GULF,6000,46.00,0,1L,SET50,2022-03-04
5,O,IVL,3000,48.00,0,P50%,SET50,2022-05-31
6,O,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
7,O,KTC,2100,59.00,0,P25%,SET50,2022-04-18
8,O,SINGER,3000,49.00,0,P25%,SET100,NaT
9,O,SSP,10000,9.35,0,6L,SET,NaT


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,11,11,11,11,11,11,11
S,9,9,9,9,9,9,9


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

20

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'ASK', 3000, 37.75, 0, '6L', 'SET', '2022-03-03']
['B', 'BBL', 2400, 125.0, 0, '1L', 'SET50', '2022-04-22']
['B', 'BCPG', 12000, 11.1, 0, 'MinP', 'SET100', 'NaT']
['B', 'GLOBAL', 9000, 20.6, 0, '1L', 'SET50', '2022-02-02']
['B', 'GULF', 6000, 46.0, 0, '1L', 'SET50', '2022-03-04']
['B', 'IVL', 3000, 48.0, 0, 'P50%', 'SET50', '2022-05-31']
['B', 'KBANK', 1500, 142.0, 0, '5pct', 'SET50', '2022-04-18']
['B', 'KTC', 2100, 59.0, 0, 'P25%', 'SET50', '2022-04-18']
['B', 'SINGER', 3000, 49.0, 0, 'P25%', 'SET100', 'NaT']
['B', 'SSP', 10000, 9.35, 0, '6L', 'SET', 'NaT']
['B', 'STANLY', 600, 172.0, 0, 'P25%', 'SET', 'NaT']
['S', 'NER', 9000, 7.1, 0, 'P50%', 'SET', '2022-04-19']
['S', 'KCE', 1000, 68.0, 0, 'C62.75', 'SET50', '2022-03-21']
['S', 'RCL', 4000, 47.5, 0, 'CP1S', 'SET100', '2022-03-23']
['S', 'MAKRO', 1500, 39.0, 0, '6M', 'SET', '2022-03-03']
['S', 'BDMS', 1500, 27.5, 0, 'RD05pct', 'SET50', '2022-03-09']
['S', 'ORI', 15000, 11.6, 0, '5pct', 'SET100', '2022-05-09']
['S', 'SIS', 3600

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,37.75,0,6L,SET,2022-03-03
1,2,B,BBL,2400,125.00,0,1L,SET50,2022-04-22
2,3,B,BCPG,12000,11.10,0,MinP,SET100,NaT
3,4,B,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
4,5,B,GULF,6000,46.00,0,1L,SET50,2022-03-04
5,6,B,IVL,3000,48.00,0,P50%,SET50,2022-05-31
6,7,B,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
7,8,B,KTC,2100,59.00,0,P25%,SET50,2022-04-18
8,9,B,SINGER,3000,49.00,0,P25%,SET100,NaT
9,10,B,SSP,10000,9.35,0,6L,SET,NaT


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,37.75,0,6L,SET,2022-03-03
1,2,B,BBL,2400,125.00,0,1L,SET50,2022-04-22
2,3,B,BCPG,12000,11.10,0,MinP,SET100,2022-02-02
3,4,B,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
4,5,B,GULF,6000,46.00,0,1L,SET50,2022-03-04
5,6,B,IVL,3000,48.00,0,P50%,SET50,2022-05-31
6,7,B,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
7,8,B,KTC,2100,59.00,0,P25%,SET50,2022-04-18
8,9,B,SINGER,3000,49.00,0,P25%,SET100,2022-02-02
9,10,B,SSP,10000,9.35,0,6L,SET,2022-02-02


In [29]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,37.75,0,6L,SET,2022-03-03
1,2,B,BBL,2400,125.00,0,1L,SET50,2022-04-22
2,3,B,BCPG,12000,11.10,0,MinP,SET100,NaT
3,4,B,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
4,5,B,GULF,6000,46.00,0,1L,SET50,2022-03-04
5,6,B,IVL,3000,48.00,0,P50%,SET50,2022-05-31
6,7,B,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
7,8,B,KTC,2100,59.00,0,P25%,SET50,2022-04-18
8,9,B,SINGER,3000,49.00,0,P25%,SET100,NaT
9,10,B,SSP,10000,9.35,0,6L,SET,NaT


In [30]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

4

In [31]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [32]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [33]:
### Process to set target price

In [34]:
### 1) Set50 records

In [35]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,2,B,BBL,2400,125.00,0,1L,SET50,2022-04-22
1,4,B,GLOBAL,9000,20.60,0,1L,SET50,2022-02-02
2,5,B,GULF,6000,46.00,0,1L,SET50,2022-03-04
3,6,B,IVL,3000,48.00,0,P50%,SET50,2022-05-31
4,7,B,KBANK,1500,142.00,0,5pct,SET50,2022-04-18
5,8,B,KTC,2100,59.00,0,P25%,SET50,2022-04-18
6,16,S,BDMS,1500,27.50,0,RD05pct,SET50,2022-03-09
7,13,S,KCE,1000,68.00,0,C62.75,SET50,2022-03-21
8,20,S,TOP,3000,60.50,0,5pct,SET50,2022-03-03
9,19,S,TTB,100000,1.33,0,5pct,SET50,2022-04-21


In [36]:
name = 'KTC'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.000000,20.000000,2.000000e+01,20.000000
mean,60.225000,60.725000,59.500000,6.586919e+06,59.987500
std,1.302578,1.180711,1.397837,4.661380e+06,1.472277
min,57.750000,58.250000,56.500000,1.733665e+06,57.000000
25%,59.000000,59.750000,58.687500,4.154750e+06,59.000000
50%,60.875000,61.375000,59.750000,4.931112e+06,60.000000
75%,61.250000,61.750000,60.750000,6.494192e+06,61.250000
max,61.750000,62.250000,61.250000,1.771117e+07,62.000000


### 2) Set100 records

In [37]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,BCPG,12000,11.1,0,MinP,SET100,2022-02-02
1,9,B,SINGER,3000,49.0,0,P25%,SET100,2022-02-02
2,17,S,ORI,15000,11.6,0,5pct,SET100,2022-05-09
3,14,S,RCL,4000,47.5,0,CP1S,SET100,2022-03-23


In [38]:
name = 'SPRC'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,10.605833,10.733333,10.400833,3.513759e+07,10.558333
std,1.243188,1.249497,1.205900,1.675251e+07,1.240563
min,8.500000,8.600000,8.350000,1.176940e+07,8.550000
25%,9.637500,9.787500,9.500000,2.252736e+07,9.587500
50%,10.150000,10.300000,9.950000,3.313813e+07,10.150000
75%,11.700000,11.800000,11.500000,4.307819e+07,11.625000
max,13.300000,13.400000,13.100000,9.231242e+07,13.200000


In [39]:
### 3) Set records

In [40]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,ASK,3000,37.75,0,6L,SET,2022-03-03
1,10,B,SSP,10000,9.35,0,6L,SET,2022-02-02
2,11,B,STANLY,600,172.00,0,P25%,SET,2022-02-02
3,15,S,MAKRO,1500,39.00,0,6M,SET,2022-03-03
4,12,S,NER,9000,7.10,0,P50%,SET,2022-04-19
5,18,S,SIS,3600,30.00,0,5pct,SET,2022-03-10


In [41]:
name = 'IP'
limit = 120 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,120.000000,120.000000,120.000000,1.200000e+02,120.000000
mean,19.886667,20.195000,19.676667,1.013248e+06,19.930833
std,0.895078,0.947252,0.849294,1.153003e+06,0.878003
min,18.300000,18.700000,17.800000,7.544500e+04,18.300000
25%,19.200000,19.400000,19.000000,3.259658e+05,19.200000
50%,19.800000,20.100000,19.600000,6.523025e+05,19.900000
75%,20.300000,20.625000,20.025000,1.121660e+06,20.300000
max,22.500000,22.900000,22.200000,6.610782e+06,22.600000


### After call Ord-log.bat

In [42]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [43]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans   name  ..., width=800)

In [44]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,AIT,-3,P25%,18000,6.50,6.65,-0.75%,1,2022-02-02
1,Buy,AP,-5,P25%,10000,10.00,10.50,-3.67%,1,2022-05-10
2,Buy,ASP,-3,RD15pct,10000,3.16,3.22,-0.62%,1,2022-02-02
3,Buy,BCPG,-7,MinP,12000,11.10,11.80,0.00%,1,2022-02-02
4,Buy,BPP,-2,MinP,9000,15.30,15.50,0.00%,1,2022-02-02
5,Buy,DIF,-1,RD05pct,10000,13.50,13.60,0.00%,2,2022-05-17
6,Buy,GULF,-9,1L,6000,46.00,48.25,+2.12%,1,2022-03-04
7,Buy,IVL,-11,P50%,3000,48.00,50.75,-1.46%,0,2022-05-31
8,Buy,KBANK,-14,5pct,1500,142.00,149.00,-0.33%,0,2022-04-18
9,Buy,RCL,-14,3L,3000,39.75,43.25,-1.70%,0,2022-02-02
